# Read PDF file

You got an internship position in a company. However, your boss didn't prepare anything for you. So he just wants you to move the data from PDF to Excel. And there are 200 pdf files that you have to do. Your boss thought it will take a couple weeks to move from the PDF to Excel. Because he never learnt programming. Fortunately, you took the Jungwon's programming class.

## Task 1 : Read the content inside of the PDF file.

![pdf image](sample-image-1.png)


The name of the pdf file is sample-PDF.pdf. Your first task is, loading the all data into the python code.

There are several options that you can choose.
I used this : https://gist.github.com/yoavram/4351598

In [1]:
# pdfx usage: http://pdfx.cs.man.ac.uk/usage
import requests
import pprint
from bs4 import BeautifulSoup
url = "http://pdfx.cs.man.ac.uk"

In [2]:
def pypdfx(filename):
    f = open(filename, 'rb')
    files = {'file': f}
    try:
        print('Sending', filename, 'to', url)
        r = requests.post(url, files=files, headers={'Content-Type':'application/pdf'})
        print('Got status code', r.status_code)
    finally:
        f.close()
    return r.content

In [ ]:
filename = 'sample_pdf.pdf'
respond = pypdfx(filename)

Sending sample_pdf.pdf to http://pdfx.cs.man.ac.uk


In [ ]:
soup = BeautifulSoup(respond, "xml")
content = soup.find('region',{"id": "5"}).contents[0]
print(content)
new_list = ["empty"]+content.split(" ")
num_of_col = content[0:content.index('January')].count(" ") + 1

data = {}

for i in range(0,len(new_list),num_of_col) :
    if i < num_of_col :
        #make keys with each countries name
        for k in range(1,num_of_col):
            data[new_list[k]]=[]
    else :
        for k in range(1,num_of_col):
            data[new_list[k]].append(int(new_list[i+k].replace(",","")))
            

pprint.pprint(data)

### Calculate the max, min, and average amount from Germany.

In [ ]:
max_val = max(data["Germany"])
print("Max is %f" % max_val)
min_val = min(data["Germany"])
print("Min is %f" % min_val)
avg_val = sum(data["Germany"])/float(len(data["Germany"]))
print("Avg is %f" % avg_val)

## Task 2 : Write the content into the Excel file.

Now you have to make the Excel file. The most important point is the last row should be the total amount of each column using SUM function.

![sample image](sample-image-2.png)


In [ ]:
import xlsxwriter
import calendar

workbook = xlsxwriter.Workbook('sample-xls.xlsx')
worksheet = workbook.add_worksheet()

row = 0


cols = list(data.keys())
worksheet.write(row, 0,"")
for i in range(len(cols)):
    worksheet.write(row, i+1, cols[i])
row += 1 

for i in range(0,12):
    worksheet.write(row, 0, calendar.month_name[i+1])
    for k, v in enumerate(cols) : 
        worksheet.write(row, k+1, data[v][i])
    row += 1


# Write a total using a formula.
worksheet.write(row, 0, 'Total')
for i in range(len(cols)):
    col = chr(ord('A')+1+i)
    func = '=SUM(%s%d:%s%d)' % (col,2,col,row)
    worksheet.write(row, i+1, func)
    

workbook.close()

## Final check point
### Will your code work for the different number of the column?